In [12]:
import pandas as pd

crashes = pd.read_csv('data/crashes_crashes.csv')

people = pd.read_csv('data/crashes_people.csv')

vehicles = pd.read_csv('data/crashes_vehicles.csv')




/var/folders/bt/mf607s9n0xs6sr08g8c44w9c0000gn/T/ipykernel_14397/2158860904.py:5: DtypeWarning: Columns (19,23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  people = pd.read_csv('data/crashes_people.csv')
/var/folders/bt/mf607s9n0xs6sr08g8c44w9c0000gn/T/ipykernel_14397/2158860904.py:7: DtypeWarning: Columns (18,20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles = pd.read_csv('data/crashes_vehicles.csv')


In [87]:
crashes_prime_cause = crashes[crashes['PRIM_CONTRIBUTORY_CAUSE'] != 'UNABLE TO DETERMINE']

crashes_prime_cause_filled = crashes_prime_cause.fillna({'INTERSECTION_RELATED_I': 'N', 'NOT_RIGHT_OF_WAY_I': 
                                                         'N', 'HIT_AND_RUN_I':'N'})

crashes_prime_cause_filleddrop = crashes_prime_cause_filled.dropna(subset=['LATITUDE', 'LONGITUDE', 'INJURIES_TOTAL', 'INJURIES_FATAL', 'MOST_SEVERE_INJURY'])


from sklearn.cluster import KMeans

n_clusters = 30  # Number of clusters to create
X = crashes_prime_cause_filleddrop[['LONGITUDE', 'LATITUDE']]

# Create a K-Means clustering model
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans.fit(X)

# Add cluster labels to your data
cluster_labels = kmeans.labels_
crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels





crashes_prime_cause_filleddrop = crashes_prime_cause_filleddrop.drop(columns = ['CRASH_DATE_EST_I', 'DEVICE_CONDITION', 'REPORT_TYPE', 'DATE_POLICE_NOTIFIED',
                                                          'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME','BEAT_OF_OCCURRENCE', 'PHOTOS_TAKEN_I', 
                                                          'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 
                                                          'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING', 'INJURIES_NO_INDICATION', 
                                                          'INJURIES_UNKNOWN', 'LOCATION', 'LATITUDE', 'LONGITUDE', 'LANE_CNT', 'CRASH_DATE', 
                                                          'INJURIES_REPORTED_NOT_EVIDENT', 'TRAFFIC_CONTROL_DEVICE', 'INJURIES_TOTAL', 
                                                          'INJURIES_FATAL'])
crashes_prime_cause_filleddrop.info()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/var/folders/bt/mf607s9n0xs6sr08g8c44w9c0000gn/T/ipykernel_14397/3962835711.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crashes_prime_cause_filleddrop['GEO_KMEANS_Cluster'] = cluster_labels


<class 'pandas.core.frame.DataFrame'>
Index: 484057 entries, 2 to 798905
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   CRASH_RECORD_ID          484057 non-null  object
 1   POSTED_SPEED_LIMIT       484057 non-null  int64 
 2   WEATHER_CONDITION        484057 non-null  object
 3   LIGHTING_CONDITION       484057 non-null  object
 4   FIRST_CRASH_TYPE         484057 non-null  object
 5   TRAFFICWAY_TYPE          484057 non-null  object
 6   ALIGNMENT                484057 non-null  object
 7   ROADWAY_SURFACE_COND     484057 non-null  object
 8   ROAD_DEFECT              484057 non-null  object
 9   CRASH_TYPE               484057 non-null  object
 10  INTERSECTION_RELATED_I   484057 non-null  object
 11  NOT_RIGHT_OF_WAY_I       484057 non-null  object
 12  HIT_AND_RUN_I            484057 non-null  object
 13  DAMAGE                   484057 non-null  object
 14  PRIM_CONTRIBUTORY_CAUSE  

In [86]:

#decide which injuiry column to drop
crashes_prime_cause_filleddrop['MOST_SEVERE_INJURY'].value_counts()
#crashes_prime_cause_filleddrop['INJURIES_TOTAL'].value_counts()
#crashes_prime_cause_filleddrop['INJURIES_FATAL'].value_counts()


MOST_SEVERE_INJURY
NO INDICATION OF INJURY     406574
NONINCAPACITATING INJURY     44287
REPORTED, NOT EVIDENT        23056
INCAPACITATING INJURY         9571
FATAL                          569
Name: count, dtype: int64

In [92]:
#Transforming to numerical data

crashes_prime_cause_filleddrop_dummy = pd.get_dummies(crashes_prime_cause_filleddrop, columns = ['WEATHER_CONDITION', 'LIGHTING_CONDITION',
                                                                                                 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 
                                                                                                 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 
                                                                                                 'SEC_CONTRIBUTORY_CAUSE', 'ROAD_DEFECT',
                                                                                                 'MOST_SEVERE_INJURY'], drop_first=True, 
                                                                                                  dtype=int)
                                                                                                 




#binary/tertiary
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in ['CRASH_TYPE','INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE']:
    crashes_prime_cause_filleddrop_dummy[col] = le.fit_transform(crashes_prime_cause_filleddrop_dummy[col])


crashes_cleaned = crashes_prime_cause_filleddrop_dummy





,CRASH_RECORD_ID,POSTED_SPEED_LIMIT,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,PRIM_CONTRIBUTORY_CAUSE,NUM_UNITS,CRASH_HOUR,...,ROAD_DEFECT_NO DEFECTS,ROAD_DEFECT_OTHER,"ROAD_DEFECT_RUT, HOLES",ROAD_DEFECT_SHOULDER DEFECT,ROAD_DEFECT_UNKNOWN,ROAD_DEFECT_WORN SURFACE,MOST_SEVERE_INJURY_INCAPACITATING INJURY,MOST_SEVERE_INJURY_NO INDICATION OF INJURY,MOST_SEVERE_INJURY_NONINCAPACITATING INJURY,"MOST_SEVERE_INJURY_REPORTED, NOT EVIDENT"
2,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,30,1,0,0,1,2,FAILING TO REDUCE SPEED TO AVOID CRASH,4,14,...,1,0,0,0,0,0,0,1,0,0
46,c107aea0fb8320ca5e7676759af9399cffe67f3ac8e429...,30,1,0,0,1,2,IMPROPER BACKING,2,17,...,1,0,0,0,0,0,0,1,0,0
56,db61137bc4bb19b84665da12c4c367592e874c623f76a1...,30,0,0,0,0,2,IMPROPER LANE USAGE,3,18,...,1,0,0,0,0,0,0,1,0,0
63,8640aa03a86f10df7a8c1dafc03d1551c4a4c0262397f3...,30,1,0,0,1,2,DRIVING SKILLS/KNOWLEDGE/EXPERIENCE,2,11,...,1,0,0,0,0,0,0,1,0,0
74,f1779af8b006d956f29723ac415bd54b61ef9acf84e38e...,30,1,1,0,1,2,FAILING TO YIELD RIGHT-OF-WAY,2,15,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798900,ffffc784918a94c6d5ec9cdced500004faafc6a9e6e01b...,30,1,0,0,0,1,NOT APPLICABLE,2,11,...,1,0,0,0,0,0,0,1,0,0
798901,ffffc802346fd6f48f99117898fbc558237a3052c327b8...,30,0,0,0,0,2,UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN...,2,7,...,1,0,0,0,0,0,0,0,0,1
798902,ffffd143cf6dc784876069140869d2d724080089450108...,30,1,0,0,1,2,IMPROPER LANE USAGE,2,19,...,0,0,0,0,1,0,0,1,0,0
798904,ffffe577251dc1cc67f2f823b595e5028427a210e2a40e...,30,1,0,0,1,1,FAILING TO YIELD RIGHT-OF-WAY,2,8,...,1,0,0,0,0,0,0,1,0,0
